In [47]:
import os
import threading
import time
import datetime, random
import pandas as pd
from tqdm.notebook import tqdm

from IPython.display import HTML
import plotly.express as px
import xml.etree.ElementTree as ET
root_node = ET.parse(file).getroot()
from bs4 import BeautifulSoup

In [7]:
dataloc = "data/images/"
outputloc = "output"
labels = "data/labels"
rdcl_path = "data/RDCL2019/"

In [11]:
all_files = []
files_list = os.listdir(rdcl_path)

Following goe shtouhg the example RDCL2019 dataset. this reads xml and extracts the raw text. then run the .tiff through documents kraken and tesseract

In [56]:
textfiles = {}
location = []
for file in files_list:
    input_file = "{}{}".format(rdcl_path, file)
    output_file_text = "{}/RDCL2019/text/{}.txt".format(outputloc, file[:-4])
    
    #we get unicode out of the xml file
    if file[-3:] == 'xml':
        fd = open(input_file, 'r')
        
        xml_file = fd.read()

        soup = BeautifulSoup(xml_file, 'lxml')
        
        od = open(output_file_text, 'w')

        for tag in soup.findAll('unicode'):
            od.write(tag.text)
            #print(tag.text)
        od.close()
    
    #run the none xml files though kraken and tesseract for later. 
    output_file_kraken = "{}/RDCL2019/kraken/{}.txt".format(outputloc, file[:-4])
    output_file_tesseract = "{}/RDCL2019/tesseract/{}.txt".format(outputloc, file[:-4])
        
    pro_time = time.process_time()
    system_cmd = "kraken -i {} {} binarize segment ocr -m en_best.mlmodel".format(input_file, output_file_kraken)
    os.system(system_cmd)
    location.append(["kraken", time.process_time() - pro_time, input_file, output_file_kraken, output_file_text])

    system_cmd = "tesseract {} {} ".format(input_file, output_file_tesseract)
    pro_time = time.process_time()
    os.system(system_cmd)
    location.append(["tesseract", time.process_time() - pro_time,input_file, output_file_tesseract, output_file_text])
    

We want statistics BLEU, Rouge n-rouge 

In [57]:
import warnings
warnings.filterwarnings('ignore')

import nltk.translate.bleu_score as bleu


In [19]:
bleu.

[['kraken', 0.0020523670000001104, 'data/RDCL2019/00770911.tif', 'output/RDCL2019/kraken/00770911.txt'], ['tesseract', 0.00023181500000024613, 'data/RDCL2019/00770911.tif', 'output/RDCL2019/tesseract/00770911.txt'], ['kraken', 0.0001892809999999301, 'data/RDCL2019/00771128.tif', 'output/RDCL2019/kraken/00771128.txt'], ['tesseract', 0.0002469300000003116, 'data/RDCL2019/00771128.tif', 'output/RDCL2019/tesseract/00771128.txt'], ['kraken', 0.0006924480000001232, 'data/RDCL2019/00770355.tif', 'output/RDCL2019/kraken/00770355.txt'], ['tesseract', 0.00018231900000031942, 'data/RDCL2019/00770355.tif', 'output/RDCL2019/tesseract/00770355.txt'], ['kraken', 0.0008255160000003592, 'data/RDCL2019/00770279.tif', 'output/RDCL2019/kraken/00770279.txt'], ['tesseract', 0.0003399110000001926, 'data/RDCL2019/00770279.tif', 'output/RDCL2019/tesseract/00770279.txt'], ['kraken', 0.000777421999999639, 'data/RDCL2019/00771226.tif', 'output/RDCL2019/kraken/00771226.txt'], ['tesseract', 0.0001424089999999545, '